In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130620

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,118878.2605
2018-02-28,134731.8561
2018-03-31,126297.6012
2018-04-30,153567.4932
2018-05-31,143512.1051
2018-06-30,150574.5186
2018-07-31,155707.0252
2018-08-31,178688.0313
2018-09-30,150642.9724


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    118878.2605
 2018-02-28    134731.8561
 2018-03-31    126297.6012
 2018-04-30    153567.4932
 2018-05-31    143512.1051
 2018-06-30    150574.5186
 2018-07-31    155707.0252
 2018-08-31    178688.0313
 2018-09-30    150642.9724
 2018-10-31    147635.7864
 2018-11-30    123834.9659
 2018-12-31    134139.4874
 2019-01-31    154159.9757
 2019-02-28    172908.0422
 2019-03-31    176478.4737
 2019-04-30    173818.6295
 2019-05-31    183571.2294
 2019-06-30    188873.8623
 2019-07-31    168079.7968
 2019-08-31    202524.5249
 2019-09-30    156683.9436
 2019-10-31    161999.0965
 2019-11-30    158866.3514
 2019-12-31    131606.3906
 2020-01-31    139282.1373
 2020-02-29     95481.7457
 2020-03-31    106772.2463
 2020-04-30     84518.1174
 2020-05-31    107403.6017
 2020-06-30     99382.1996
 2020-07-31    157628.7679
 2020-08-31    142502.5867
 2020-09-30    107737.8832
 2020-10-31    135939.7601
 2020-11-30    126060.4241
 2020-12-31    103974.6034

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.639050
p-value : 0.085192
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -8.828766
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=984.158, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=973.897, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=968.986, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=968.953, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=964.998, Time=0.04 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=962.376, Time=0.39 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=957.683, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=958.474, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=958.444, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=960.617, Time=0.07 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=962.265, Time=0.09 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=957.523, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=959.062, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=960.057, Time=0.14 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(3,0,0)(0,0,0)[1] inter

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -474.751
Method:                       css-mle   S.D. of innovations          19453.642
Date:                Sat, 11 Sep 2021   AIC                            957.503
Time:                        04:47:58   BIC                            964.454
Sample:                             0   HQIC                           960.051
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.402e+05   1.14e+04     12.322      0.000    1.18e+05    1.62e+05
ar.L1.y        0.5360      0.149      3.609      0.000       0.245       0.827
ar.L2.y        0.2223      0.148      1.501      0.1

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([144703.30389346, 144105.24521658]), array([19453.64186233, 22072.03412856]), array([[106574.86647515, 182831.74131177],
       [100844.85325907, 187365.63717409]]))
